In [ ]:
! pip install yfinance
! pip install --upgrade yfinance
! pip install fuzzywuzzy[speedup]
import yfinance as yf
import pandas as pd
import datetime
import numpy as np
from fuzzywuzzy import fuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.3 MB/s eta 0:00:00


In [ ]:
# Define the period
start_date = datetime.date.today() - datetime.timedelta(days=5*365)
start_date = start_date.strftime("%Y-%m-%d")
end_date = datetime.date.today().strftime("%Y-%m-%d")

# Define the tickers
ticker_symbols = ["AAPL","TSLA","JNJ","NVDA","AIR.PA","SIE.DE","OR.PA","NESN.SW","TCS.NS",
                  "VALE","BABA","NIO","VOO","VEA","VWO","ARKK","FBALX","VBIAX","TLT","BND",
                  "IEI","SHV","GLD","BTC-USD"]

all_data = pd.DataFrame()
volatility_data = []

# Loop through the tickers
for ticker_symbol in ticker_symbols:
    try:
        data = yf.download(ticker_symbol, start=start_date, end=end_date, progress=False)

        if not data.empty:
            data['Daily_Return'] = data['Close'].pct_change()

            # Calculate annual volatility and return
            annual_volatility = data['Daily_Return'].std() * np.sqrt(252)
            annual_return = data['Daily_Return'].mean() * 252

            #print(f"{ticker_symbol} - Volatility: {annual_volatility:.4f}, Return: {annual_return:.4f}")

            # Store info for later filtering
            volatility_data.append({
                "Ticker": ticker_symbol,
                "Annual_Volatility": annual_volatility,
                "Annual_Return": annual_return
            })

            # Merge closing prices into all_data
            data = data.rename(columns={'Close': f'{ticker_symbol}_Close'})
            if all_data.empty:
                all_data = data[[f'{ticker_symbol}_Close']]
            else:
                all_data = pd.merge(all_data, data[[f'{ticker_symbol}_Close']], left_index=True, right_index=True, how='outer')
        else:
            print(f"No data found for ticker: {ticker_symbol}")

    except Exception as e:
        print(f"Error fetching data for {ticker_symbol}: {e}")

# Remove rows with missing values
all_data = all_data.dropna()
all_data.to_csv('daily_prices_combined.csv')
#print("Combined daily prices saved to daily_prices_combined.csv")

# Function to map user profile to a volatility filter
def map_user_risk_to_vol_filter(risk_profile):
    risk_profile = risk_profile.lower()
    if risk_profile == "conservative":
        return lambda vol: vol < 0.2
    elif risk_profile == "balanced":
        return lambda vol: 0.2 <= vol < 0.4
    elif risk_profile == "aggressive":
        return lambda vol: vol >= 0.4
    else:
        return lambda vol: False

# Function to request user info
def get_user_input():
    print("\n--- Investment Profile ---")

    # Loop to validate the amount
    while True:
        try:
            amount = float(input("Amount to invest (€): "))
            if amount > 0:
                break
            else:
                print("Please enter a positive amount.")
        except ValueError:
            print("Please enter a valid number.")

    # Loop to validate the risk profile
    while True:
        risk_profile = input("Risk profil (Conservative, Balanced, Aggressive): ").lower()
        options = ["conservative", "balanced", "aggressive"]
        best_match = max(options, key=lambda x: fuzz.ratio(risk_profile, x))
        if fuzz.ratio(risk_profile, best_match) > 80:  # Similarity threshold
            risk_profile = best_match
            break
        else:
            print("Invalid risk profile. Please choose from Conservative, Balanced or Aggressive.")

    # Loop to validate the horizon with spelling correction
    while True:
        horizon = input("Horizon (Short, Medium, Long): ").lower()
        options = ["short", "medium", "long"]
        best_match = max(options, key=lambda x: fuzz.ratio(horizon, x))
        if fuzz.ratio(horizon, best_match) > 80:  # Similarity threshold
            horizon = best_match
            break
        else:
            print("Invalid horizon. Please choose from Short, Medium or Long.")


    # Loop to validate the desired return
    while True:
        try:
            desired_return = float(input("Desired return (% per year): "))
            if desired_return > 0:
                break
            else:
                print("Please enter a positive return.")
        except ValueError:
            print("Please enter a valid number.")

    return amount, risk_profile, horizon, desired_return


    #amount = float(input("Montant à investir (€): "))
    #risk_profile = input("Profil de risque (Conservative, Balanced, Aggressive): ").lower()
    #horizon = input("Horizon (Short, Medium, Long): ").lower()
    #desired_return = float(input("Rendement souhaité (% par an): "))
    #return amount, risk_profile, horizon, desired_return



# Determine how many products to offer based on the amount
def determine_num_products(amount):
    l = [[10_000, 5], [100_000, 7], [500_000, 10]]
    num_products = 3  # default value
    for i in range(len(l)-1, -1, -1):
        if l[i][0] <= amount:
            num_products = l[i][1]
            break
    return num_products


# Suggest products based on profile and amount
def suggest_products(volatility_data, risk_profile, amount):
    vol_filter = map_user_risk_to_vol_filter(risk_profile)
    filtered = [item for item in volatility_data if vol_filter(item["Annual_Volatility"])]

    # Sort by decreasing return
    filtered.sort(key=lambda x: x["Annual_Return"], reverse=True)

    # Number of products to offer
    num_products = determine_num_products(amount)

    # Calculate average volatility and average return
    avg_volatility = np.mean([p["Annual_Volatility"] for p in filtered[:num_products]])
    avg_return = np.mean([p["Annual_Return"] for p in filtered[:num_products]])

    print(f"\n Recommendations for a {risk_profile.capitalize()} avec un investissement de {amount:.0f} € :")
    if filtered:
        for p in filtered[:num_products]:
            print(f"- {p['Ticker']} | Volatility: {p['Annual_Volatility']:.2%} | Annual Return: {p['Annual_Return']:.2%}")
    else:
        print("No product matches your risk profile.")

    # Display average volatility and average return
    print(f"\n Average portfolio volatility : {avg_volatility:.2%}")
    print(f"Average portfolio return : {avg_return:.2%}")

# Execution
if __name__ == "__main__":
    while True:  # Beginning of the while loop
        amount, risk_profile, horizon, desired_return = get_user_input()
        suggest_products(volatility_data, risk_profile, amount)

        another_simulation = input("Do you want to perform another simulation? (Yes/No): ").lower()
        if another_simulation != "yes":
            break  # Exit the while loop if the user does not want to restart




--- Investment Profile ---

 Recommendations for a Conservative avec un investissement de 50000 € :
- VOO | Volatility: 17.73% | Annual Return: 16.63%
- GLD | Volatility: 15.13% | Annual Return: 14.06%
- VEA | Volatility: 16.93% | Annual Return: 12.88%
- FBALX | Volatility: 12.95% | Annual Return: 11.77%
- VWO | Volatility: 18.27% | Annual Return: 10.20%

 Average portfolio volatility : 16.20%
Average portfolio return : 13.11%
